In [13]:
%load_ext autoreload
%autoreload 2

from datasets import load_from_disk
import pickle
import torch
from torch import utils
import pytorch_lightning as pl

from nli.data import DataSetPadding
from nli.models import AvgWordEmb, UniLSTM, BiLSTM, MaxPoolLSTM, MLP
from nli.learner import NLINet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
with open('store/wordvec.pkl', 'rb') as f:
    wordvec = pickle.load(f)
dataset_snli = load_from_disk('data/snli')

In [15]:
dataloader = {}
for split, shuffle in zip(['train', 'validation'], [True, False]):
    dataset = DataSetPadding(dataset_snli[split], wordvec)
    if split == 'train':
        dataset = utils.data.Subset(dataset, range(10000))
    dataloader[split] = utils.data.DataLoader(dataset, batch_size=64, shuffle=shuffle, num_workers=3)

In [16]:
# encoder = AvgWordEmb()
# classifier = MLP(300*4, 512, 3)

# encoder = UniLSTM()
# classifier = MLP(2048*4, 512, 3)

# encoder = BiLSTM()
# classifier = MLP(2048*4*2, 512, 3)

encoder = MaxPoolLSTM()
classifier = MLP(2048*4*2, 512, 3)


model = NLINet(encoder, classifier)

device = 'gpu' if torch.cuda.is_available() else 'cpu'
trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1, accelerator=device, enable_checkpointing=False)
trainer.fit(model, train_dataloaders = dataloader['train'], val_dataloaders = dataloader['validation'])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type             | Params
------------------------------------------------
0 | encoder    | MaxPoolLSTM      | 38.5 M
1 | classifier | MLP              | 8.7 M 
2 | loss_fn    | CrossEntropyLoss | 0     
------------------------------------------------
47.2 M    Trainable params
0         Non-trainable params
47.2 M    Total params
188.623   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

u shape torch.Size([64, 4096])
u shape torch.Size([64, 4096])


Training: 0it [00:00, ?it/s]

u shape torch.Size([64, 4096])
